In [81]:

import time
import datetime
import pandas as pd

ticker = 'AAPL'
period1 = int(time.mktime(datetime.datetime(2010, 12, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2022, 12, 1, 23, 59).timetuple()))
interval = '1d' # 1d, 1m

query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

df = pd.read_csv(query_string)
# print(df)
df.to_csv('AAPL.csv')

In [75]:
from statsmodels.tsa.stattools import adfuller
def ad_test(dataset):
     dftest = adfuller(dataset, autolag = 'AIC')
     print("1. ADF : ",dftest[0])
     print("2. P-Value : ", dftest[1])
     print("3. Num Of Lags : ", dftest[2])
     print("4. Num Of Observations Used For ADF Regression:",      dftest[3])
     print("5. Critical Values :")
     for key, val in dftest[4].items():
         print("\t",key, ": ", val)
ad_test(df['Close'])

1. ADF :  -1.3415990743508153
2. P-Value :  0.6099324564594122
3. Num Of Lags :  11
4. Num Of Observations Used For ADF Regression: 995
5. Critical Values :
	 1% :  -3.4369391965679257
	 5% :  -2.86444914142694
	 10% :  -2.5683189679553546


In [82]:
from pmdarima import auto_arima
stepwise_fit = auto_arima(df['Close'], trace=True,
suppress_warnings=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=10790.079, Time=3.91 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=10810.067, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=10801.017, Time=0.12 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=10800.307, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=10811.033, Time=0.07 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=10801.060, Time=1.33 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=10801.019, Time=1.72 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=10795.513, Time=4.39 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=10793.660, Time=2.35 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=10799.259, Time=0.79 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=10801.643, Time=1.33 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=10801.157, Time=2.18 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=10791.629, Time=5.07 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=inf, Time=2.06 sec

Best model:  ARIMA(2,1,2

In [ ]:
pip install pmdarima

In [77]:
print(df.shape)
train=df.iloc[:-30]
test=df.iloc[-30:]
print(train.shape,test.shape)

(1007, 7)
(977, 7) (30, 7)


In [83]:
from statsmodels.tsa.arima.model import ARIMA
model=ARIMA(train['Close'],order=(2,1,2))
model=model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  Close   No. Observations:                  977
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -2214.920
Date:                Wed, 04 Jan 2023   AIC                           4439.839
Time:                        11:46:38   BIC                           4464.257
Sample:                             0   HQIC                          4449.130
                                - 977                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.0029      0.091     11.049      0.000       0.825       1.181
ar.L2         -0.8452      0.076    -11.068      0.000      -0.995      -0.695
ma.L1         -1.0504      0.090    -11.639      0.000      -1.227      -0.873
ma.L2          0.8547      0.076     11.294      0.000       0.706       1.003
sigma2         5.4784      0.168     32.676      0.000       5.150       5.807
===================================================================================
Ljung-Box (L1) (Q):                   0.59   Jarque-Bera (JB):               257.55
Prob(Q):                              0.44   Prob(JB):                         0.00
Heteroskedasticity (H):               4.57   Skew:                            -0.24
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.47
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
pip install statsmodels

In [84]:
start=len(train)
end=len(train)+len(test)-1
pred=model.predict(start=start,end=end,typ='levels').rename('ARIMA Predictions')
pred

977     144.134588
978     144.183247
979     143.999981
980     143.775053
981     143.704350
982     143.823537
983     144.002831
984     144.081920
985     144.009711
986     143.870448
987     143.791802
988     143.830624
989     143.936027
990     144.008930
991     143.992966
992     143.915341
993     143.850979
994     143.852034
995     143.907486
996     143.962211
997     143.970231
998     143.932023
999     143.886926
1000    143.873986
1001    143.899123
1002    143.935270
1003    143.950279
1004    143.934782
1005    143.906555
1006    143.891342
Name: ARIMA Predictions, dtype: float64

In [70]:
from sklearn.metrics import mean_squared_error
from math import sqrt
test['Close'].mean()
rmse=sqrt(mean_squared_error(pred,test['Close']))
print((rmse/test['Close'].mean()) * 100)

3.6698964199248736


In [93]:
import pandas as pd
import statsmodels.api as sm

# Load the data into a Pandas dataframe
df = pd.read_csv("AAPL.csv")

# Fit an ARIMA model to the time series data
model = sm.tsa.arima.ARIMA(df["Close"], order=(2,1,1))
model_fit = model.fit()

# Make predictions for the next 5 time steps
predictions = model_fit.predict(start=len(df), end=len(df)+5)
predictions

3022    148.110358
3023    148.054734
3024    148.043119
3025    148.040889
3026    148.040473
3027    148.040397
Name: predicted_mean, dtype: float64

Ecnomic Indicaters

In [98]:

pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
